# VOLTANDO UMA PASTA PARA TRAZ

In [1]:
import os
os.chdir("../")

# DOWNLOAD DOS ARQUIVOS

In [ ]:
from modules.ComexStatDownloader import ComexStatDownloader

CSD = ComexStatDownloader()
CSD.download_files()

# CONEXÃO COM BANCO

In [ ]:
from modules.database.postgres_connection import PostgresConnection
from modules.database.database_manager import DatabaseManager

host    = "localhost"
db      = "comexstat"
user    = "postgres"
pwd     = "postgres"  # Substitua pela sua senha real

# Certifique-se de definir as variáveis host, db, user, pwd antes de usar
conn = PostgresConnection(host, db, user, pwd)
conn.connect()
db_manager = DatabaseManager(conn)



# CRIANDO TABELAS NO BANCO

In [3]:
# Cria as tabelas no banco de dados
with open("repository\\querys\\create_table.sql", "r") as file:
    create_tables_sql = file.read()
    db_manager.execute_raw(create_tables_sql)

# CARREGANDO DADOS NO BANCO

In [7]:
from jinja2 import Template
import pandas as pd
import os

# Insere os dados no banco de dados
base_paths = ["data\\external\\mun", "data\\external\\ncm"]
batch_size = 30000
years       = ["2023"]     

for base_path in base_paths:
    for paths in os.listdir(base_path):
        path_end                = os.path.join(base_path, paths)
        for file_path in os.listdir(path_end):
            if file_path.endswith(".csv"):
                
                year = "".join([n for n in os.path.basename(file_path) if n.isdigit()])
                if year not in years:
                    continue
                
                # Lê o arquivo CSV
                file_path       = os.path.join(path_end, file_path)
                df              = pd.read_csv(file_path, sep=";", encoding="latin1", low_memory=False)
                columns         = df.columns
                print(f"Arquivo {file_path} lido com sucesso.")
                
                # Cria o cabeçalho da query de inserção
                columns_lower   = [col.lower() for col in columns.to_list()]
                placeholders    = ", ".join(["%s"] * len(columns_lower))
                tabela          = f"{os.path.basename(base_path).lower()}_{os.path.basename(path_end).lower()}"
                template        = Template("INSERT INTO {{ tabela }} ({{ cols|join(', ') }}) VALUES ({{ placeholders }})")
                query_header    = template.render(tabela=tabela, cols=columns_lower, placeholders=placeholders)
                
                # Inserção em batch de x em x linhas
                rows            = df.to_numpy()
                for i in range(0, len(rows), batch_size):
                    batch       = rows[i:i+batch_size]
                    params      = [tuple([str(item) for item in row]) for row in batch]
                    db_manager.create_batch(query_header, params)
                    print(f"Lote de linhas {i} a {i+len(batch)-1} inserido com sucesso.")


Arquivo data\external\mun\EXP\2023.csv lido com sucesso.
Lote de linhas 0 a 29999 inserido com sucesso.
Lote de linhas 30000 a 59999 inserido com sucesso.
Lote de linhas 60000 a 89999 inserido com sucesso.
Lote de linhas 90000 a 119999 inserido com sucesso.
Lote de linhas 120000 a 149999 inserido com sucesso.
Lote de linhas 150000 a 179999 inserido com sucesso.
Lote de linhas 180000 a 209999 inserido com sucesso.
Lote de linhas 210000 a 239999 inserido com sucesso.
Lote de linhas 240000 a 269999 inserido com sucesso.
Lote de linhas 270000 a 299999 inserido com sucesso.
Lote de linhas 300000 a 329999 inserido com sucesso.
Lote de linhas 330000 a 359999 inserido com sucesso.
Lote de linhas 360000 a 389999 inserido com sucesso.
Lote de linhas 390000 a 419999 inserido com sucesso.
Lote de linhas 420000 a 449999 inserido com sucesso.
Lote de linhas 450000 a 479999 inserido com sucesso.
Lote de linhas 480000 a 509999 inserido com sucesso.
Lote de linhas 510000 a 539999 inserido com sucesso.
